In [1]:
import pandas as pd 
import wandb
from datetime import datetime 
import pathlib 
import plotly.express as px

In [2]:
api = wandb.Api()
runs = api.runs(
    "anrg-iobt_ns/iobt_ns",
    filters={"createdAt": {"$gte": "2021-10-26T08:00:00"}}
) 

IGNORE_KEYS = {"metadata", "mother_network", "network"}

records = []
for run in runs: 
    records.append({
        **{
            k: v for k, v in run.summary.items() 
            if not k.startswith("_") and k not in IGNORE_KEYS
        },
        **{
            k: v for k, v in run.config.items() 
            if not k.startswith("_") and k not in IGNORE_KEYS
        }
    })

df = pd.DataFrame.from_records(records)

In [3]:
results = ["best_cost"]
variables = [
    col for col in df.columns 
    if col not in {"cost", "best_cost", "iteration"} and len(df[col].unique()) > 1
]

df = df.drop(columns=set(df.columns) - set(results).union(variables))

df_mean: pd.DataFrame = df.groupby(variables).mean().reset_index()
df_median: pd.DataFrame = df.groupby(variables).median().reset_index()
df_std: pd.DataFrame = df.groupby(variables).std().reset_index()

df = df.sort_values(by=variables)
df


,best_cost,sa_base,optimizer,network_order,sa_neighborhood,task_graph_order,sa_initial_temperature
84,217.70,0.0,simulated_annealing,10,random_connected_neighbor,10,10.0
81,241.66,0.0,simulated_annealing,10,random_connected_neighbor,10,50.0
78,2303.02,0.0,simulated_annealing,10,random_connected_neighbor,10,100.0
65,153.64,0.0,simulated_annealing,10,random_connected_neighbor,50,10.0
62,2254.16,0.0,simulated_annealing,10,random_connected_neighbor,50,50.0
...,...,...,...,...,...,...,...
85,115.96,NaN,random,10,NaN,10,NaN
66,131.96,NaN,random,10,NaN,50,NaN
47,154.36,NaN,random,10,NaN,100,NaN
28,13071.44,NaN,random,50,NaN,10,NaN


In [8]:
import numpy as np

df["sa_base"] = df["sa_base"].apply(lambda x: "e" if x == np.e else f"{x:.2f}")
px.line(
    df[df["optimizer"] == "simulated_annealing"],
    x="network_order",
    y="best_cost",
    color="sa_neighborhood",
    facet_col="sa_initial_temperature",
    facet_row="sa_base",
    animation_frame="task_graph_order",

    template="simple_white",
    labels={
        "sa_base": "Base",
        "best_cost": "Cost",
        "sa_initial_temperature": "T=0 Temp",
        "network_order": "# Nodes",
        "task_graph_order": "# Tasks",
        "sa_neighborhood": "Neighborhood Function"
    }
)